In [1]:
!nvidia-smi

!pip install -q timm scikit-learn
!pip install --upgrade timm


Sun Nov 30 20:55:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os, math, time, random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

import torchvision
from torchvision import transforms
import timm
from tqdm.auto import tqdm
from sklearn.neighbors import KNeighborsClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Seeds
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# ---------- CONFIG: ViT-Tiny + big batch ----------
CFG = {
    "image_size": 128,          # all crops -> 128x128
    "batch_size": 256,          # if OOM -> 192 or 160
    "num_workers": 0,           # avoid multiprocessing issues
    "epochs_pretrain": 100,      # self-supervised DINO epochs
    "epochs_linear": 150,        # linear probe epochs
    "learning_rate_dino": 5e-4,
    "learning_rate_linear": 0.1,
    "weight_decay": 1e-5,
    "warmup_epochs": 2,
    "out_dim": 256,             # DINO projection dim
    "student_temp": 0.1,
    "teacher_temp": 0.04,
    "momentum_teacher": 0.996,
}
print(CFG)


Device: cuda
{'image_size': 128, 'batch_size': 256, 'num_workers': 0, 'epochs_pretrain': 100, 'epochs_linear': 150, 'learning_rate_dino': 0.0005, 'learning_rate_linear': 0.1, 'weight_decay': 1e-05, 'warmup_epochs': 2, 'out_dim': 256, 'student_temp': 0.1, 'teacher_temp': 0.04, 'momentum_teacher': 0.996}


In [3]:
CIFAR_MEAN = (0.5071, 0.4867, 0.4408)
CIFAR_STD  = (0.2675, 0.2565, 0.2761)

class DinoMultiCropTransform:
    """
    2 global crops + 2 local crops.
    All crops OUTPUT at size = CFG["image_size"] (128x128),
    locals differ only by scale range.
    """
    def __init__(self, size=128, n_global=2, n_local=2):
        self.n_global = n_global
        self.n_local = n_local
        self.size = size

        flip_color = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
        ])

        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
        ])

        self.global_transforms = [
            transforms.Compose([
                transforms.RandomResizedCrop(
                    size, scale=(0.4, 1.0),
                    interpolation=transforms.InterpolationMode.BICUBIC,
                ),
                flip_color,
                transforms.GaussianBlur(kernel_size=15, sigma=(0.1, 2.0)),
                normalize,
            ]),
            transforms.Compose([
                transforms.RandomResizedCrop(
                    size, scale=(0.4, 1.0),
                    interpolation=transforms.InterpolationMode.BICUBIC,
                ),
                flip_color,
                transforms.GaussianBlur(kernel_size=15, sigma=(0.1, 2.0)),
                transforms.RandomSolarize(threshold=0.5, p=0.2),
                normalize,
            ]),
        ]

        # local crops: smaller scale, but still resized to 128
        self.local_transform = transforms.Compose([
            transforms.RandomResizedCrop(
                size, scale=(0.08, 0.4),
                interpolation=transforms.InterpolationMode.BICUBIC,
            ),
            flip_color,
            transforms.GaussianBlur(kernel_size=9, sigma=(0.1, 2.0)),
            normalize,
        ])

    def __call__(self, x):
        crops = []
        for i in range(self.n_global):
            crops.append(self.global_transforms[i](x))
        for _ in range(self.n_local):
            crops.append(self.local_transform(x))
        return crops

# ---- Datasets ----
dino_transform = DinoMultiCropTransform(size=CFG["image_size"], n_global=2, n_local=2)

train_unlabeled = torchvision.datasets.CIFAR100(
    root="./data", train=True, download=True, transform=dino_transform
)

linear_train_transform = transforms.Compose([
    transforms.Resize(CFG["image_size"],
                     interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

linear_test_transform = transforms.Compose([
    transforms.Resize(CFG["image_size"],
                     interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

train_labeled = torchvision.datasets.CIFAR100(
    root="./data", train=True, download=False, transform=linear_train_transform
)
test_labeled = torchvision.datasets.CIFAR100(
    root="./data", train=False, download=False, transform=linear_test_transform
)

# ---- DataLoaders ----
pretrain_loader = DataLoader(
    train_unlabeled,
    batch_size=CFG["batch_size"],
    shuffle=True,
    num_workers=CFG["num_workers"],
    pin_memory=True,
    drop_last=True,
)

linear_train_loader = DataLoader(
    train_labeled,
    batch_size=CFG["batch_size"],
    shuffle=True,
    num_workers=CFG["num_workers"],
    pin_memory=True,
)

linear_test_loader = DataLoader(
    test_labeled,
    batch_size=CFG["batch_size"],
    shuffle=False,
    num_workers=CFG["num_workers"],
    pin_memory=True,
)

print("Unlabeled train:", len(train_unlabeled))
print("Labeled train:", len(train_labeled))
print("Labeled test :", len(test_labeled))


100%|██████████| 169M/169M [00:03<00:00, 47.7MB/s]


Unlabeled train: 50000
Labeled train: 50000
Labeled test : 10000


In [4]:
class VitBackbone(nn.Module):
    def __init__(self, model_name="vit_tiny_patch16_224", img_size=128):
        super().__init__()
        self.vit = timm.create_model(
            model_name,
            pretrained=False,
            num_classes=0,
            img_size=img_size,
        )

    def forward(self, x):
        return self.vit(x)


class DinoHead(nn.Module):
    def __init__(self, in_dim, out_dim=256, hidden_dim=2048, bottleneck_dim=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, bottleneck_dim),
            nn.GELU(),
        )
        self.last_layer = nn.utils.weight_norm(
            nn.Linear(bottleneck_dim, out_dim, bias=False)
        )
        self.last_layer.weight_g.data.fill_(1.0)
        self.last_layer.weight_g.requires_grad = False

    def forward(self, x):
        x = self.mlp(x)
        x = F.normalize(x, dim=-1, p=2)
        return self.last_layer(x)


def build_dino_vit(out_dim=256, img_size=128):
    backbone = VitBackbone("vit_tiny_patch16_224", img_size=img_size)
    embed_dim = backbone.vit.num_features
    head = DinoHead(embed_dim, out_dim=out_dim)
    return backbone, head

student_backbone, student_head = build_dino_vit(
    out_dim=CFG["out_dim"],
    img_size=CFG["image_size"],
)
student = nn.Sequential(student_backbone, student_head).to(device)

teacher_backbone, teacher_head = build_dino_vit(
    out_dim=CFG["out_dim"],
    img_size=CFG["image_size"],
)
teacher = nn.Sequential(teacher_backbone, teacher_head).to(device)
for p in teacher.parameters():
    p.requires_grad = False

print("Student (ViT-Tiny) params (M):",
      sum(p.numel() for p in student.parameters()) / 1e6)


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Student (ViT-Tiny) params (M): 6.484672


In [5]:
class DINOLoss(nn.Module):
    def __init__(self, out_dim, student_temp=0.1, teacher_temp=0.04, center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.teacher_temp = teacher_temp
        self.center_momentum = center_momentum
        self.register_buffer("center", torch.zeros(1, out_dim))

    def forward(self, student_out, teacher_out):
        student_out = [s / self.student_temp for s in student_out]
        teacher_out = [(t - self.center) / self.teacher_temp for t in teacher_out]

        student_out = [F.log_softmax(s, dim=-1) for s in student_out]
        teacher_out = [F.softmax(t, dim=-1).detach() for t in teacher_out]

        total_loss, n_terms = 0.0, 0
        for t in teacher_out:
            for s in student_out:
                loss = torch.sum(-t * s, dim=-1).mean()
                total_loss += loss
                n_terms += 1
        total_loss /= n_terms

        batch_center = torch.cat(teacher_out).mean(dim=0, keepdim=True)
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)
        return total_loss

def update_teacher(student, teacher, m):
    with torch.no_grad():
        for ps, pt in zip(student.parameters(), teacher.parameters()):
            pt.data.mul_(m).add_(ps.data, alpha=1.0 - m)

dino_loss_fn = DINOLoss(
    out_dim=CFG["out_dim"],
    student_temp=CFG["student_temp"],
    teacher_temp=CFG["teacher_temp"],
).to(device)


In [6]:
def cosine_scheduler(base_value, final_value, epochs, niter_per_epoch,
                     warmup_epochs=0, start_warmup_value=0.0):
    iters = epochs * niter_per_epoch
    warmup_iters = warmup_epochs * niter_per_epoch
    schedule = []
    for i in range(iters):
        if i < warmup_iters:
            v = start_warmup_value + i / max(1, warmup_iters) * (base_value - start_warmup_value)
        else:
            progress = (i - warmup_iters) / max(1, iters - warmup_iters)
            v = final_value + 0.5 * (base_value - final_value) * (1.0 + math.cos(math.pi * progress))
        schedule.append(v)
    return schedule

params = [p for p in student.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(
    params,
    lr=CFG["learning_rate_dino"],
    weight_decay=CFG["weight_decay"],
)

niter_per_epoch = len(pretrain_loader)
lr_schedule = cosine_scheduler(
    CFG["learning_rate_dino"],
    1e-6,
    CFG["epochs_pretrain"],
    niter_per_epoch,
    warmup_epochs=CFG["warmup_epochs"],
)

momentum_schedule = cosine_scheduler(
    CFG["momentum_teacher"],
    1.0,
    CFG["epochs_pretrain"],
    niter_per_epoch,
)

print("iters/epoch:", niter_per_epoch, "total iters:", len(lr_schedule))


iters/epoch: 195 total iters: 19500


In [7]:
scaler = GradScaler()

def train_dino(student, teacher, dino_loss_fn, loader, optimizer,
               lr_schedule, momentum_schedule, epochs):
    it = 0
    student.train()
    teacher.eval()

    for epoch in range(epochs):
        epoch_loss = 0.0
        start = time.time()
        pbar = tqdm(loader, desc=f"[DINO] Epoch {epoch+1}/{epochs}")

        for crops, _ in pbar:
            crops = [c.to(device, non_blocking=True) for c in crops]

            for pg in optimizer.param_groups:
                pg["lr"] = lr_schedule[it]
            m = momentum_schedule[it]

            with autocast(dtype=torch.float16):
                student_out = [student(c) for c in crops]
                with torch.no_grad():
                    teacher_out = [teacher(c) for c in crops[:2]]  # 2 global crops
                loss = dino_loss_fn(student_out, teacher_out)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            update_teacher(student, teacher, m)

            epoch_loss += loss.item()
            it += 1
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        epoch_loss /= len(loader)
        print(f"Epoch {epoch+1}: loss={epoch_loss:.4f}, time={time.time()-start:.1f}s")

    return student, teacher


/tmp/ipython-input-2193493272.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [8]:
student, teacher = train_dino(
    student,
    teacher,
    dino_loss_fn,
    pretrain_loader,
    optimizer,
    lr_schedule,
    momentum_schedule,
    CFG["epochs_pretrain"],
)

os.makedirs("checkpoints", exist_ok=True)
torch.save(student_backbone.state_dict(),
           "checkpoints/dino_vit_tiny_cifar100_backbone_batch256.pth")
print("Backbone saved.")


[DINO] Epoch 1/100:


100%|██████████████████████████████████████████████████████████ 195/195 [14:00<00:16, 4.76it/s]



Epoch   1: loss=4.9490, time=945.0s

[DINO] Epoch 2/100:


100%|██████████████████████████████████████████████████████████ 195/195 [16:00<00:16, 4.79it/s]



Epoch   2: loss=4.9317, time=938.9s

[DINO] Epoch 3/100:


...


Epoch  98: loss=0.8806, time=963.3s

[DINO] Epoch 99/100:


100%|██████████████████████████████████████████████████████████ 195/195 [16:00<00:15, 4.58it/s]



Epoch  99: loss=0.8418, time=946.0s

[DINO] Epoch 100/100:


100%|██████████████████████████████████████████████████████████ 195/195 [14:00<00:00, 4.76it/s]


Epoch 100: loss=0.8157, time=973.7s



In [ ]:
backbone_linear = VitBackbone(
    "vit_tiny_patch16_224",
    img_size=CFG["image_size"],
).to(device)

state_dict = torch.load(
    "checkpoints/dino_vit_tiny_cifar100_backbone_batch256.pth",
    map_location=device,
)
backbone_linear.load_state_dict(state_dict)
backbone_linear.eval()
for p in backbone_linear.parameters():
    p.requires_grad = False

num_classes = 100

class LinearProbe(nn.Module):
    def __init__(self, backbone, in_dim, num_classes=100):
        super().__init__()
        self.backbone = backbone
        self.fc = nn.Linear(in_dim, num_classes)

    def forward(self, x):
        with torch.no_grad():
            feat = self.backbone(x)
        return self.fc(feat)

embed_dim = backbone_linear.vit.num_features
linear_model = LinearProbe(backbone_linear, embed_dim, num_classes=num_classes).to(device)

criterion_ce = nn.CrossEntropyLoss()
optimizer_linear = torch.optim.SGD(
    linear_model.fc.parameters(),
    lr=CFG["learning_rate_linear"],
    momentum=0.9,
    weight_decay=0.0,
)

def evaluate_linear(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            logits = model(imgs)
            _, preds = logits.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total if total > 0 else 0.0

def train_linear(model, train_loader, test_loader, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        correct = total = 0

        pbar = tqdm(train_loader, desc=f"[Linear] Epoch {epoch+1}/{epochs}")
        for imgs, labels in pbar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer_linear.zero_grad()
            logits = model(imgs)
            loss = criterion_ce(logits, labels)
            loss.backward()
            optimizer_linear.step()

            total_loss += loss.item()
            _, preds = logits.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

            pbar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "train_acc": f"{100.0 * correct / total:.2f}%"
            })

        avg_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        test_acc = evaluate_linear(model, test_loader)
        print(f"Epoch {epoch+1}: loss={avg_loss:.4f}, train_acc={train_acc:.2f}%, test_acc={test_acc:.2f}%")


In [11]:
train_linear(
    linear_model,
    linear_train_loader,
    linear_test_loader,
    CFG["epochs_linear"],
)


[Linear] Epoch 1/150:


100%|█████████████████████████████████████████████████████ 196/196 [00:46<00:00, 4.64it/s]



Epoch   1: loss=3.8733, train_acc=10.40%, test_acc=12.63%

[Linear] Epoch 2/150:


100%|█████████████████████████████████████████████████████ 196/196 [00:44<00:00, 4.56it/s]



Epoch   2: loss=3.8387, train_acc=10.81%, test_acc=12.81%

[Linear] Epoch 3/150:


...


Epoch 148: loss=1.2490, train_acc=78.07%, test_acc=76.45%

[Linear] Epoch 149/150:


100%|█████████████████████████████████████████████████████ 196/196 [00:40<00:00, 4.57it/s]



Epoch 149: loss=1.2000, train_acc=78.49%, test_acc=76.62%

[Linear] Epoch 150/150:


100%|█████████████████████████████████████████████████████ 196/196 [00:46<00:00, 4.64it/s]


Epoch 150: loss=1.2000, train_acc=78.71%, test_acc=76.80%



In [12]:
def extract_features(backbone, loader):
    backbone.eval()
    feats, labels_all = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Extracting features"):
            imgs = imgs.to(device)
            f = backbone(imgs)
            feats.append(f.cpu())
            labels_all.append(labels)
    feats = torch.cat(feats, dim=0).numpy()
    labels_all = torch.cat(labels_all, dim=0).numpy()
    return feats, labels_all

train_feats, train_labels = extract_features(backbone_linear, linear_train_loader)
test_feats,  test_labels  = extract_features(backbone_linear, linear_test_loader)

knn = KNeighborsClassifier(n_neighbors=20, metric="cosine")
knn.fit(train_feats, train_labels)
knn_acc = knn.score(test_feats, test_labels)
print(f"k-NN accuracy on CIFAR-100 (ViT-Tiny DINO features): {knn_acc * 100:.2f}%")

Extracting features: 100%|██████████████████████████████████████████ 196/196 [00:44<00:00, 4.48it/s]
Extracting features: 100%|████████████████████████████████████████████ 40/40 [00:07<00:00, 4.62it/s]



k-NN accuracy on CIFAR-100 (ViT-Tiny DINO features): 76.34%
